Source
https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py

In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
import keras
import data_functions
import numpy
#Load data from already created pickle file
words, X, dataX, dataY, n_words, n_vocab, index2word, word2index = data_functions.load_from_pickle("data.pickle")

In [3]:
print(index2word[1])
print(index2word[n_vocab-1])

من
يوسوس


In [4]:
#Uncomment and Run this is plot_model does not work
import os
os.environ["PATH"] =  os.environ["PATH"] + os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin'

def show_image(imagefile, w, h):
    %pylab inline
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    img=mpimg.imread(imagefile)
    plt.figure(figsize=(w, h))
    imgplot = plt.imshow(img);
    plt.show();

def show_model(model, w, h):
    from keras.utils.vis_utils import plot_model
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True);
    show_image('model_plot.png', w, h)

In [ ]:
class GAN():
    def __init__(self):
#         self.img_rows = 28
#         self.img_cols = 28
#         self.channels = 1
#         self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.input_shape = 100
        self.noise_dim = 50
        self.img_shape = (self.input_shape,1)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.noise_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (self.noise_dim,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

#         print(model.summary())
#         show_model(model, 20, 20)

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

#         img_shape = (self.img_rows, self.img_cols, self.channels)
        img_shape = self.img_shape

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))   #, input_shape=img_shape
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
#         print(model.summary())
#         show_model(model, 10, 10)

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128, save_interval=50):

        # Load the dataset
#         (X_train, _), (_, _) = mnist.load_data()

#         # Rescale -1 to 1
#         X_train = (X_train.astype(np.float32) - 127.5) / 127.5
#         X_train = np.expand_dims(X_train, axis=3)
#         print('X_train shape=',X_train.shape)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
#             print('imgs shape=',imgs.shape)

            noise = np.random.normal(0, 1, (half_batch, self.noise_dim))
#             print('noise shape=',noise.shape)
            
            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r = 5
        noise = np.random.normal(0, 1, (r , self.noise_dim))
        gen_imgs = self.generator.predict(noise)
        pred = np.reshape(gen_imgs * float(n_vocab), (r,-1))
        pred[pred < 0] = 0
        pred[pred > (n_vocab-1)] = 0
        pred = pred.astype(int)        
        for p in pred:
            s = [index2word[i+1] for i in p]
            s = ' '.join(s)
            print(s)
            print('----------------------')

#         print(pred)

#         # Rescale images 0 - 1
#         gen_imgs = 0.5 * gen_imgs + 0.5

#         fig, axs = plt.subplots(r, c)
#         cnt = 0
#         for i in range(r):
#             for j in range(c):
#                 axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
#                 axs[i,j].axis('off')
#                 cnt += 1
#         fig.savefig("./images/mnist_%d.png" % epoch)
#         plt.close()


In [ ]:
gan = GAN()

In [ ]:
gan.train(X[0:200], epochs=30000, batch_size=32, save_interval=200)

/root/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.656479, acc.: 53.12%] [G loss: 0.911361]
من من يتولوا من من من نجب من من من فليأتكم وذلكم وأثابهم بلحيتي من قبلتك من من من برسوله من من من خلفكم وانحر من من من من من من فاستغلظ فليتنافس عمرك من ركاب الموقدة وهمت من من تذكرون من من من خامدين من مكبا من من من من سيصلى من قيام من من من من آنية من وتحبون والأنف من من يوسوس عصفا من من من من الجنات فيحل غاوين من من من من من أيهم من من من والغاوون من من فتنتكم من والإيمان يضيفوهما أحاديث ترجعونها ينكرونها واغفر شهد عائلا من من والصابرين من أسرفوا
----------------------
من أقطارها يكونون من من عرضا من من الحزبين من بيانه وتؤتوها من فتعاطى من ونورهم سبقا من من منتشر يجعلوه تمور من صاغرين يزدهم من من من الأكرم من من نتبعهم من ستدعون من من خلقته يجزى من من من بذنوبنا من صوما سرحوهن أحصى من من من من ويلقون بفاكهة الزاد يبعث من دعاءكم من من ملاقيكم من وبنيه من يعصون من قوا تزرعون يسجرون فقلت إرم عبدا الأحمال لعنتم مقتكم من من من من من الشتاء من جبارا تحملون ومضى من وأخرج من من ونمنعكم من من من من من وعميانا من لخلق من قدحا ذرية من
----

109 [D loss: 0.310636, acc.: 100.00%] [G loss: 3.396820]
110 [D loss: 0.286595, acc.: 100.00%] [G loss: 3.319979]
111 [D loss: 0.278833, acc.: 100.00%] [G loss: 3.139539]
112 [D loss: 0.286089, acc.: 100.00%] [G loss: 3.094206]
113 [D loss: 0.291240, acc.: 100.00%] [G loss: 3.266034]
114 [D loss: 0.251536, acc.: 100.00%] [G loss: 3.394944]
115 [D loss: 0.284454, acc.: 96.88%] [G loss: 3.291351]
116 [D loss: 0.252319, acc.: 100.00%] [G loss: 3.366156]
117 [D loss: 0.271626, acc.: 100.00%] [G loss: 3.549044]
118 [D loss: 0.270443, acc.: 100.00%] [G loss: 3.693529]
119 [D loss: 0.256819, acc.: 96.88%] [G loss: 4.072772]
120 [D loss: 0.253178, acc.: 100.00%] [G loss: 4.026159]
121 [D loss: 0.254817, acc.: 100.00%] [G loss: 3.847982]
122 [D loss: 0.253278, acc.: 100.00%] [G loss: 3.696590]
123 [D loss: 0.239388, acc.: 100.00%] [G loss: 3.948114]
124 [D loss: 0.237409, acc.: 100.00%] [G loss: 3.917588]
125 [D loss: 0.223823, acc.: 100.00%] [G loss: 3.775464]
126 [D loss: 0.228657, acc.: 100.

209 [D loss: 0.061419, acc.: 100.00%] [G loss: 4.461366]
210 [D loss: 0.074140, acc.: 100.00%] [G loss: 4.584688]
211 [D loss: 0.062741, acc.: 100.00%] [G loss: 4.695498]
212 [D loss: 0.065544, acc.: 100.00%] [G loss: 4.574060]
213 [D loss: 0.064156, acc.: 100.00%] [G loss: 4.607646]
214 [D loss: 0.067636, acc.: 100.00%] [G loss: 4.374702]
215 [D loss: 0.059417, acc.: 100.00%] [G loss: 4.450063]
216 [D loss: 0.069573, acc.: 100.00%] [G loss: 4.752496]
217 [D loss: 0.060051, acc.: 100.00%] [G loss: 5.005921]
218 [D loss: 0.060334, acc.: 100.00%] [G loss: 4.931567]
219 [D loss: 0.057157, acc.: 100.00%] [G loss: 4.782652]
220 [D loss: 0.055174, acc.: 100.00%] [G loss: 4.634476]
221 [D loss: 0.055837, acc.: 100.00%] [G loss: 4.242789]
222 [D loss: 0.053574, acc.: 100.00%] [G loss: 4.532684]
223 [D loss: 0.059703, acc.: 100.00%] [G loss: 4.497542]
224 [D loss: 0.057332, acc.: 100.00%] [G loss: 4.535241]
225 [D loss: 0.057837, acc.: 100.00%] [G loss: 4.731030]
226 [D loss: 0.057865, acc.: 10

359 [D loss: 0.103464, acc.: 100.00%] [G loss: 3.520258]
360 [D loss: 0.158001, acc.: 90.62%] [G loss: 3.027449]
361 [D loss: 0.151263, acc.: 100.00%] [G loss: 3.684689]
362 [D loss: 0.095316, acc.: 100.00%] [G loss: 3.558571]
363 [D loss: 0.134843, acc.: 100.00%] [G loss: 3.616519]
364 [D loss: 0.153609, acc.: 93.75%] [G loss: 3.433771]
365 [D loss: 0.094500, acc.: 100.00%] [G loss: 3.430766]
366 [D loss: 0.136379, acc.: 100.00%] [G loss: 3.952595]
367 [D loss: 0.155579, acc.: 96.88%] [G loss: 3.615169]
368 [D loss: 0.127931, acc.: 100.00%] [G loss: 3.934695]
369 [D loss: 0.119867, acc.: 100.00%] [G loss: 3.690104]
370 [D loss: 0.051266, acc.: 100.00%] [G loss: 3.301209]
371 [D loss: 0.113424, acc.: 96.88%] [G loss: 3.348915]
372 [D loss: 0.158852, acc.: 93.75%] [G loss: 4.142566]
373 [D loss: 0.088686, acc.: 100.00%] [G loss: 4.560565]
374 [D loss: 0.156267, acc.: 96.88%] [G loss: 3.626669]
375 [D loss: 0.137128, acc.: 93.75%] [G loss: 4.097589]
376 [D loss: 0.212855, acc.: 90.62%] [

...

4259 [D loss: 0.329244, acc.: 87.50%] [G loss: 3.404133]
4260 [D loss: 0.245604, acc.: 96.88%] [G loss: 7.932310]
4261 [D loss: 0.447238, acc.: 71.88%] [G loss: 3.428777]
4262 [D loss: 0.257003, acc.: 90.62%] [G loss: 3.617393]
4263 [D loss: 0.495127, acc.: 78.12%] [G loss: 2.041580]
4264 [D loss: 0.268564, acc.: 96.88%] [G loss: 3.444471]
4265 [D loss: 0.408323, acc.: 93.75%] [G loss: 2.956848]
4266 [D loss: 0.452054, acc.: 75.00%] [G loss: 1.672922]
4267 [D loss: 0.299233, acc.: 100.00%] [G loss: 2.943589]
4268 [D loss: 0.167688, acc.: 100.00%] [G loss: 3.362905]
4269 [D loss: 0.321913, acc.: 90.62%] [G loss: 2.637887]
4270 [D loss: 0.388404, acc.: 84.38%] [G loss: 1.492854]
4271 [D loss: 0.421050, acc.: 84.38%] [G loss: 1.714121]
4272 [D loss: 0.254600, acc.: 96.88%] [G loss: 2.501723]
4273 [D loss: 0.446016, acc.: 87.50%] [G loss: 2.066348]
4274 [D loss: 0.322231, acc.: 87.50%] [G loss: 3.123609]
4275 [D loss: 0.373800, acc.: 87.50%] [G loss: 3.446209]
4276 [D loss: 0.411413, acc.:

4403 [D loss: 0.204702, acc.: 93.75%] [G loss: 4.496921]
4404 [D loss: 0.256399, acc.: 93.75%] [G loss: 4.517184]
4405 [D loss: 0.340806, acc.: 81.25%] [G loss: 2.987775]
4406 [D loss: 0.181112, acc.: 96.88%] [G loss: 4.471447]
4407 [D loss: 0.381880, acc.: 78.12%] [G loss: 2.100086]
4408 [D loss: 0.204237, acc.: 96.88%] [G loss: 4.282917]
4409 [D loss: 0.283028, acc.: 90.62%] [G loss: 3.347586]
4410 [D loss: 0.352514, acc.: 90.62%] [G loss: 4.065736]
4411 [D loss: 0.270200, acc.: 93.75%] [G loss: 4.559480]
4412 [D loss: 0.346532, acc.: 87.50%] [G loss: 2.568003]
4413 [D loss: 0.401447, acc.: 81.25%] [G loss: 2.485333]
4414 [D loss: 0.218298, acc.: 100.00%] [G loss: 5.730368]
4415 [D loss: 0.308141, acc.: 93.75%] [G loss: 3.230580]
4416 [D loss: 0.342082, acc.: 87.50%] [G loss: 5.032738]
4417 [D loss: 0.270106, acc.: 100.00%] [G loss: 6.384313]
4418 [D loss: 0.253035, acc.: 96.88%] [G loss: 3.218373]
4419 [D loss: 0.372293, acc.: 78.12%] [G loss: 1.997778]
4420 [D loss: 0.514415, acc.:

4555 [D loss: 0.294115, acc.: 96.88%] [G loss: 2.081140]
4556 [D loss: 0.222607, acc.: 96.88%] [G loss: 7.643967]
4557 [D loss: 0.186982, acc.: 93.75%] [G loss: 5.652993]
4558 [D loss: 0.366598, acc.: 84.38%] [G loss: 2.783509]
4559 [D loss: 0.176467, acc.: 100.00%] [G loss: 5.456795]
4560 [D loss: 0.193661, acc.: 96.88%] [G loss: 4.087039]
4561 [D loss: 0.217356, acc.: 93.75%] [G loss: 5.009227]
4562 [D loss: 0.283243, acc.: 93.75%] [G loss: 3.270588]
4563 [D loss: 0.207526, acc.: 96.88%] [G loss: 6.749932]
4564 [D loss: 0.298593, acc.: 87.50%] [G loss: 5.130735]
4565 [D loss: 0.249938, acc.: 93.75%] [G loss: 5.239376]
4566 [D loss: 0.271180, acc.: 84.38%] [G loss: 2.443533]
4567 [D loss: 0.187621, acc.: 96.88%] [G loss: 4.810990]
4568 [D loss: 0.290402, acc.: 84.38%] [G loss: 3.642702]
4569 [D loss: 0.194506, acc.: 96.88%] [G loss: 3.211294]
4570 [D loss: 0.168119, acc.: 93.75%] [G loss: 6.986238]
4571 [D loss: 0.224115, acc.: 93.75%] [G loss: 4.084642]
4572 [D loss: 0.348015, acc.: 

4656 [D loss: 0.356492, acc.: 81.25%] [G loss: 4.664755]
4657 [D loss: 0.166140, acc.: 100.00%] [G loss: 5.954325]
4658 [D loss: 0.277312, acc.: 96.88%] [G loss: 3.458180]
4659 [D loss: 0.225154, acc.: 90.62%] [G loss: 6.570867]
4660 [D loss: 0.243335, acc.: 93.75%] [G loss: 2.514792]
4661 [D loss: 0.173543, acc.: 100.00%] [G loss: 4.714722]
4662 [D loss: 0.241863, acc.: 93.75%] [G loss: 2.543559]
4663 [D loss: 0.302557, acc.: 90.62%] [G loss: 1.930804]
4664 [D loss: 0.396714, acc.: 78.12%] [G loss: 2.826468]
4665 [D loss: 0.342215, acc.: 87.50%] [G loss: 4.278661]
4666 [D loss: 0.205213, acc.: 100.00%] [G loss: 6.739679]
4667 [D loss: 0.305203, acc.: 96.88%] [G loss: 6.799515]
4668 [D loss: 0.212700, acc.: 100.00%] [G loss: 4.838886]
4669 [D loss: 0.151073, acc.: 100.00%] [G loss: 5.427398]
4670 [D loss: 0.197428, acc.: 100.00%] [G loss: 3.986736]
4671 [D loss: 0.434447, acc.: 75.00%] [G loss: 2.952123]
4672 [D loss: 0.339180, acc.: 96.88%] [G loss: 3.020790]
4673 [D loss: 0.281617, a

4799 [D loss: 0.293865, acc.: 93.75%] [G loss: 3.403803]
4800 [D loss: 0.299810, acc.: 93.75%] [G loss: 2.799611]
لهم من حين من المخلصين من من من الملائكة آتيناكم ساحر كمن فانظر عنا تجدوه من بهم من من كارهون فعلوه من حسنا من من من المحراب أيهم الصالحات من ظهورها من الرجس يأتينكم قد من يسيرا المدينة حاشرين تطوع سبلا من من وقومه يقيمون من بذات أجورهم من تسألهم حجارة من من وجاهدوا نكذب قياما لجهنم يجادل من من الطاغوت من بمثل المخلصين من من المصير والإنس أحياها من يسيروا تدعون سيئة عملكم قيما وزكريا من يعقوب معه من أعد قديرا جنتان مصر بإذنه أواب من وأملي أتى وذروا يقوم يظلم أرى ماله ونذيرا قليل من ويقتلون شاعر أجلا
----------------------
يدعو من من وبما من نجزي من أندادا وأنتم لشديد التواب من وكنتم أرسلناك ويكون فاليوم تسألون يتول نفورا أكون الآخرة التواب لعنت من من وكتبه يخزيه وأصحاب وسبح تسألون يعمر من من أحد يغفر وليعلم غيره الإيمان وزر من حسيبا من من فإنها مؤمن يأتوا أعينهم من حبطت قريب فاستعذ من المشرق تخرجون زوجين تعدنا صبرا من لطيف من من شجرة من تحشرون من نفسه لربك التي من مثنى الصا